<h2>Retrieve satellite data using Google Earth Engine (GEE) corresponding to field data compiled by the Cape Cod Commission (CCC)</h2>

<h4>Code accompanying "Satellite imagery as a management tool for monitoring water clarity across freshwater ponds in Cape Cod, Massachusetts" (Coffer et al., 2024, <em>Journal of Environmental Management</em>). 
Python code written by co-author Nikolay Nezlin.</h4>

* Read and plot the geodatabase with pond locations from the Cape Cod Commission (CCC)  
* Read the provided CCC field data to a dataframe 
* Make a Google Earth Engine (GEE) feature collection from the field data dataframe with points and polygons
* Get mean and standard deviation of top of atmosphere (TOA) reflectances from satellite imagery using GEE 
    * For circular regions of certain radius  
    * Removing pixels within a certain distance from shore to reduce adjacency effects
* Return the feature collection, convert to a dataframe, and save as a CSV file

**Step 1: Load all required packages. If a package has not yet been installed, run "conda install [package name]" from Anaconda Prompt.**

In [ ]:
import os
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
import geemap
import datetime
import pytz
import folium
import ee

**Step 2: Initialize GEE and change the project directory to the folder where the "CCC_Ponds_Export_2020.gdb" geodatabase and the "SDD_all_samples.csv" field dataset are stored. To initialize GEE, change 'your-gee-project' to the name of your GEE project. Then, the following code will open an internet window through which you must provide all permissions and copy the code generated on the last screen. Copy that wherever your code editor requests it (for Visual Studio code, the box is at the top of the window); once this is done once, the project should be initialized for future code execution. <br /><br />This is the only section of the code that should need to be updated by the user.**

In [ ]:
# Change the following paths  
proj_dir = '...'
ccc_ponds_fn = 'Input_Data/Pond_Geodatabase/CCC_Ponds_Export_2020.gdb'
sdd_dir = 'Input_Data/Field_Data/SDD_all_samples.csv'
out_dir = 'Input_Data/Field_Data/'

# Import and initialize Earth Engine package
try:
  # Trigger the authentication flow.
  ee.Authenticate()
  ee.Initialize(project='capecod-sdd')
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

**Step 3: Run this cell to define all needed functions for the remainder of the script. Note, running this cell won't produce any output, but will set the script up to be able to run the remainder of the code.**

In [ ]:
# Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
        if ee.image.Image in [type(x) for x in v.values()]:
            folium.TileLayer(
                tiles = v["tile_fetcher"].url_format,
                attr  = 'Google Earth Engine',
                overlay =True,
                name  = k
            ).add_to(mapViz)
        else:
            folium.GeoJson(
                data = v,
                name = k
            ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

# Transform EE date to str ```YYYYMMdd'T'HHmmss```, which wiill be used in file name
def date_ee2str(date_ee, date_format="YYYYMMdd'T'HHmmss"):
    date_str = ee.Date(date_ee).format(date_format).getInfo()
    return date_str

# Transform EE date to python ```datetime```
def date_ee2datetime(date_ee):
    date_time = datetime.datetime.fromtimestamp(date_ee/1000, tz=pytz.UTC)
    return date_time

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Retrieve cloud mask data for each cloud mask type  
def img_mask_L5789(image):
    QA_RADSAT = image.select('QA_RADSAT')
    radsatMask = QA_RADSAT.bitwiseAnd(0b1111111111).eq(0)
    QA_PIXEL = image.select('QA_PIXEL')
    fillMask = QA_PIXEL.bitwiseAnd(1<<0).eq(0)  # Image data
    dilatedCloudMask = QA_PIXEL.bitwiseAnd(1<<1).eq(0)  # Dilated Cloud
    cirrusMask = QA_PIXEL.bitwiseAnd(1<<2).eq(0)  # Cirrus
    cloudMask = QA_PIXEL.bitwiseAnd(1<<3).eq(0)  # Cloud
    cloudShadowMask = QA_PIXEL.bitwiseAnd(1<<4).eq(0)  # Cloud shadow
    maskTot = radsatMask.And(dilatedCloudMask).And(cirrusMask).And(cloudMask).And(cloudShadowMask).rename('mask')
    img_masked = image.updateMask(maskTot)
    #
    return img_masked

# Mask Sentinel-2 data using opaqueCloudsMask and cirrusCloudsMask
def img_mask_S2(image):
    if ('QA60' in qa_bands_list):
        QA60 = image.select('QA60')
        opaqueCloudsMask = QA60.bitwiseAnd(1<<10).eq(0)
        cirrusCloudsMask = QA60.bitwiseAnd(1<<11).eq(0)
    else:
        opaqueCloudsMask = ee.Image(1)
        cirrusCloudsMask = ee.Image(1)
    #
    maskTot = opaqueCloudsMask.And(cirrusCloudsMask).rename('mask')
    img_masked = image.updateMask(maskTot)
    #
    return img_masked

# Apply cloud mask filtering 
def make_imgDatesDf(eeImgColl, offset2disp, count2disp):
    imgDates_df = pd.DataFrame()
    ImgCount = eeImgColl.size().getInfo()
    if (offset2disp >= ImgCount):
        return imgDates_df
    if (count2disp + offset2disp > ImgCount):
        count2disp = ImgCount - offset2disp
    if (count2disp <= 0):
        return imgDates_df
    eeImgColl10 = ee.ImageCollection(eeImgColl.toList(count2disp, offset2disp))
    imgDates_df['date_ee'] = eeImgColl10.aggregate_array("system:time_start").getInfo()
    imgDates_df['date_time'] = [date_ee2datetime(date_ee) for date_ee in imgDates_df['date_ee']]
    if (sat_code in ['L5','L7','L8','L9']):
        imgDates_df['CLOUD_COVER'] = [eeImgColl10.filterDate(int(imgDates_df['date_ee'].iat[k_img])).first(). \
                               get('CLOUD_COVER').getInfo() for k_img in range(len(imgDates_df))]
    elif (sat_code == 'S2'):
        imgDates_df['CLOUD_COVER'] = [eeImgColl10.filterDate(int(imgDates_df['date_ee'].iat[k_img])).first(). \
                               get('CLOUDY_PIXEL_PERCENTAGE').getInfo() for k_img in range(len(imgDates_df))]
    else:
        imgDates_df['CLOUD_COVER'] = -99
    return imgDates_df

# Display images
def disp_images(eeImgColl, imgDates_df, stnCentroid, bands4rgb_dict, img_scale, stnFc):
    vis_params = {
        'bands': list(bands4rgb_dict.values()),
        'min': [0,0,0],
        'max': [1/img_scale,1/img_scale,1/img_scale]}
    # Create a folium map object.
    my_map = folium.Map(location = (stnCentroid[1],stnCentroid[0]), \
                    zoom_start=8, width=500,height=400)
    # Add the the images to the map object.
    for k_image in range(len(imgDates_df)):
        ee_im1 = eeImgColl.filterDate(int(imgDates_df['date_ee'].iat[k_image])).first()
        date_str = date_ee2str(int(imgDates_df['date_ee'].iat[k_image]), date_format="YYYY-MM-dd")
        cloud_cover = imgDates_df["CLOUD_COVER"].iat[k_image]
        my_map.add_ee_layer(ee_im1, vis_params, '%s %.1f' % (date_str, cloud_cover))

    folium.GeoJson(data = stnFc.getInfo(), name='stations').add_to(my_map)
    # Add a lat lon popup.
    folium.LatLngPopup().add_to(my_map)
    # Add a layer control panel to the map.
    my_map.add_child(folium.LayerControl())
    # Display the map.
    display(my_map)

# Remove the images with duplicated dates
# From: https://gis.stackexchange.com/questions/336257/filter-out-duplicate-sentinel-2-images-form-earth-engine-image-collection-by-dat
def detectar_duplicador(image):
    esduplicado = ee.String("")
    numero = eeImgCollList.indexOf(image)
    image1 = ee.Image(eeImgCollList.get(numero.add(1)))
    # Compare the image(0) in the ImageCollection with the image(1) in the List
    fecha1 = image.date().format("Y-M-d")
    fecha2 = image1.date().format("Y-M-d")
    estado = ee.Algorithms.IsEqual(fecha1,fecha2)
    esduplicado = ee.String(ee.Algorithms.If(estado, "duplicate", "no duplicate"))
    return image.set({"duplicate": esduplicado})

# Retrieve satellite data for each relevant sensor 
def set_satellite(sat_code):
    if (sat_code == 'L5'):
        # Landsat-5 L1 (TOA)
        # 1984-03-16T16:18:01Z–2012-05-05T17:54:06
        ee_product = "LANDSAT/LT05/C02/T1_TOA"
        bands_list = ['B1', 'B2', 'B3', 'B4', 'B5','B7']  # Optical bands
        qa_bands_list = ['QA_PIXEL','QA_RADSAT'] # Do not use 'SR_CLOUD_QA'! LEDAPS quality conditions are wrong! 
        img_scale = 1
        img_offset = 0
        scale_m = 30  # Resolution for B1-B7 from "LANDSAT_LC05_C02_T1_TOA"
        bands4rgb_dict = {'Red':'B3', 'Green':'B2', 'Blue':'B1'}
        save_dir = os.path.join(proj_dir, out_dir)
    elif (sat_code == 'L7'):
        # Landsat-7 L1 (TOA)
        # 1999-05-28T01:02:17Z–present
        ee_product = "LANDSAT/LE07/C02/T1_TOA"
        bands_list = ['B1', 'B2', 'B3', 'B4', 'B5','B7']  # Optical bands
        qa_bands_list = ['QA_PIXEL','QA_RADSAT']
        img_scale = 1
        img_offset = 0
        scale_m = 30  # Resolution for B1-B5 from "LANDSAT_LC07_C02_T1_TOA"
        bands4rgb_dict = {'Red':'B4', 'Green':'B3', 'Blue':'B2'}
        save_dir = os.path.join(proj_dir, out_dir)
    elif (sat_code == 'L8'):
        # Landsat-8 L1 (TOA)
        # 2013-03-18T15:58:14Z–present
        ee_product = "LANDSAT/LC08/C02/T1_TOA"
        bands_list = ['B1', 'B2', 'B3', 'B4', 'B5','B6','B7']  # Optical bands
        qa_bands_list = ['QA_PIXEL','QA_RADSAT']
        img_scale = 1
        img_offset = 0
        scale_m = 30  # Resolution for B1-B7 from "LANDSAT_LC08_C02_T1_TOA"
        bands4rgb_dict = {'Red':'B4', 'Green':'B3', 'Blue':'B2'}
        save_dir = os.path.join(proj_dir, out_dir)
    elif (sat_code == 'L9'):
        # Landsat-9 L1 (TOA)
        # 2021-10-31T00:00:00Z–present
        ee_product = "LANDSAT/LC09/C02/T1_TOA"
        bands_list = ['B1', 'B2', 'B3', 'B4', 'B5','B6','B7']  # Optical bands
        qa_bands_list = ['QA_PIXEL','QA_RADSAT']
        img_scale = 1
        img_offset = 0
        scale_m = 30  # Resolution for B1-B7 from "LANDSAT_LC09_C02_T1_TOA"
        bands4rgb_dict = {'Red':'B4', 'Green':'B3', 'Blue':'B2'}
        save_dir = os.path.join(proj_dir, out_dir)
    elif (sat_code == 'S2'):    
        # Sentinel-2 L1 (TOA)
        # 2017-03-28T00:00:00Z - present
        ee_product = "COPERNICUS/S2_HARMONIZED"  # L1C-TOA
        bands_list = ['B1', 'B2', 'B3', 'B4', 'B5','B6','B7','B8','B8A','B9','B10','B11','B12']  # Optical bands
        qa_bands_list = ['QA60'] 
        img_scale = 0.0001
        img_offset = 0
        scale_m = 10  
        bands4rgb_dict = {'Red':'B4', 'Green':'B3', 'Blue':'B2'}
        save_dir = os.path.join(proj_dir, out_dir)
    else:
        print('Wrong satellite code %s' % sat_code)
        ee_product = ""  
        bands_list = []  # Optical bands
        qa_bands_list = [] 
        img_scale = 1
        img_offset = 0
        scale_m = 30  
        bands4rgb_dict = {}
        save_dir = ''
    return ee_product,bands_list,qa_bands_list,img_scale,img_offset,scale_m,bands4rgb_dict,save_dir

# Compute average Rrs 
def calcRrsIn(feature):
    feature = feature.set({'RrsIn': feature.propertyNames().contains(bands_list[0]+'_mean')})
    return feature

# Add product ID as a variable 
def add_product_id(feature):
    date_range = ee.Date(feature.get('date')).advance(feature.get('lagDays_mean'),'day').getRange('day')
    productIdStr = ee.Algorithms.If(condition=ee.String(sat_code).equals('S2'), \
                                    trueCase='PRODUCT_ID', falseCase='LANDSAT_PRODUCT_ID')
    PRODUCT_ID = eeImgColl.filterDate(date_range.start(),date_range.end()).first().get(productIdStr)
    return feature.set({'PRODUCT_ID': PRODUCT_ID})

# Calculate mean and Std.Dev of Rrs of <bands_list> in the circular region around the station location
def calc_RrsMeanStd(feature):
    # Filter the images collection within the date range
    dateRange4stn = ee.DateRange(ee.Date(feature.get('date')).advance(-delta_days,'day'), \
                    ee.Date(feature.get('date')).advance(delta_days,'day'))
    # Add to each image a band 'days_lag' and sort the collection 
    def add_days_lag(image):
        lagDays = ee.Number(ee.Date(image.get("system:time_start")) \
                     .difference(start=ee.Date(feature.get('date')), unit='day'))
        lagAbsDays = lagDays.abs()
        image = image.set({'lagAbsDays':lagAbsDays}) \
            .addBands(ee.Image.constant(lagDays).toFloat().rename('lagDays'))
        return image
    #
    imgColl4stn = imgCollMasked.filterDate(dateRange4stn.start(), dateRange4stn.end()) \
                .filterBounds(feature.geometry()) \
                .map(detectar_duplicador).filter(ee.Filter.eq("duplicate","no duplicate")) \
                .map(add_days_lag).sort('lagAbsDays',False) 
    #
    reducerMeanStDev = ee.Reducer.mean().combine(ee.Reducer.stdDev(),'',True)
    sampleGeom = ee.Geometry.Point([feature.getNumber('longitude'), feature.getNumber('latitude')]) \
                                    .buffer(bufferR, maxError=1) \
                    .intersection(feature.geometry().buffer(bufferS, maxError=1), maxError=1)
    feature_means = imgColl4stn.select(bands_list + ['lagDays']) \
                .mosaic().reduceRegion(**{ \
                      'reducer': reducerMeanStDev, \
                      'geometry': sampleGeom, \
                      'scale': scale_m, \
                      'maxPixels': 1e19 \
                })
    return feature.set(feature_means)

# Calculate the number of images available for each sample
def calcNimg(feature):
    dateRange4stn = ee.DateRange(ee.Date(feature.get('date')).advance(-delta_days,'day'), \
                    ee.Date(feature.get('date')).advance(delta_days,'day'))
    imgColl4stn = imgCollMasked.filterDate(dateRange4stn.start(), dateRange4stn.end()) \
                .filterBounds(feature.geometry()) \
                .map(detectar_duplicador).filter(ee.Filter.eq("duplicate","no duplicate"))
    feature = feature.set({'nImg': imgColl4stn.size()})
    return feature

**Step 4: Read in input data, including the CCC geodatabase and field data CSV file**

In [ ]:
# Read in geodatabase 
ccc_ponds_gdf = gpd.read_file(os.path.join(proj_dir,ccc_ponds_fn))
# Drop the objects without CCC_GIS_ID and Shape_Area
ccc_ponds_gdf.dropna(axis=0, how='any', subset=['CCC_GIS_ID','Shape_Area'], inplace=True)
ccc_ponds_gdf['CCC_GIS_ID'] = [s.strip() for s in ccc_ponds_gdf['CCC_GIS_ID']]
ccc_ponds_gdf = ccc_ponds_gdf.loc[ccc_ponds_gdf['CCC_GIS_ID']!='']
# Sort the objects
ccc_ponds_gdf.sort_values(by='Shape_Area', inplace=True, ascending=False, ignore_index=True)
# Transform to EPSG:4326
ccc_ponds_gdf = ccc_ponds_gdf.to_crs('EPSG:4326')
# Transform multipolygons to polygons
ccc_ponds_gdf=ccc_ponds_gdf.explode(index_parts=True)
# Read in field shapefile 
stn_df = pd.read_csv(os.path.join(proj_dir, sdd_dir))
stn_df['date'] = pd.to_datetime(stn_df['date'])

**Step 5: For each satellite, extract spectral information corresponding to field data and export coincident data as a CSV file. First, set parameters used for each satellite sensor.**

In [ ]:
# The buffer around each sampling point (m)
bufferR = 10
# The buffer offshore (m, set to less than 0)
bufferS = -30
# The number of stations to process
nStn2process = 10 
# The maximum time lag between sample and image
delta_days = 30  

**<em>Sentinel-2 (S2) which was launched 28 March 2017 and is still operational.</em>** 

In [ ]:
# Set the satellite code and start and end dates
sat_code = 'S2'; date_start = '2017-03-28'; date_end = '2024-01-01'
stn_dfSat = stn_df.loc[(stn_df['date']>=pd.to_datetime(date_start))& \
                     (stn_df['date']<=pd.to_datetime(date_end))].reset_index(drop=True).copy()
nStn = len(stn_dfSat)
# Subdirectory and file names
save_fn = 'S2_Rrs0010C.csv'
# Copy geodatabase with columns of interest 
ccc_ponds_gdf1 = ccc_ponds_gdf[['CCC_GIS_ID','geometry']].copy()
# Set satellite parameters
ee_product,bands_list,qa_bands_list,img_scale,img_offset,scale_m,bands4rgb_dict,save_dir = set_satellite(sat_code)
colNames = ['PRODUCT_ID','CCC_GIS_ID','day_str','SDD (m)','lagDays_mean','lagDays_stdDev','latitude','longitude'] \
    + list(np.array([[s+'_mean',s+'_stdDev'] for s in bands_list]).flatten())
# Extract Rrs
indx_start = 0; indx_end = indx_start+nStn2process
rrs_df = pd.DataFrame()
while (indx_start<nStn):
    stn_df1 = stn_dfSat[indx_start:indx_end].copy()
    stn_gdf = gpd.GeoDataFrame(stn_df1.merge(ccc_ponds_gdf1, how='left', left_on='CCC_GIS_ID', \
                                        right_on='CCC_GIS_ID').dropna())
    print('Processing observations %d - %d of %d (%d stations)' % (indx_start, indx_end, nStn, len(stn_gdf)))
    # Create feature collection
    stnFCwPolygons = geemap.geopandas_to_ee(stn_gdf, date='day_str', date_format='YYYY-MM-dd')
    eeImgColl = ee.ImageCollection(ee_product) \
      .filterDate(ee.Date(str(np.min(stn_gdf['date']).strftime('%Y-%m-%d'))).advance(-delta_days,'day'), \
              ee.Date(str(np.max(stn_gdf['date']).strftime('%Y-%m-%d'))).advance(delta_days,'day')) \
      .filterBounds(stnFCwPolygons) \
      .sort("system:time_start") \
      .select(bands_list + qa_bands_list)
    # Apply cloud mask 
    imgCollMasked = eeImgColl.map(lambda image: img_mask_S2(image)).select(bands_list) \
        .map(lambda image: image.multiply(img_scale).add(img_offset).copyProperties(image, ['system:time_start']))
    # Generate a List to compare dates
    eeImgCollList = eeImgColl.toList(eeImgColl.size())
    image = ee.Image(eeImgCollList.get(0))
    # Add in the end of the list a dummy image
    eeImgCollList = eeImgCollList.add(image)
    stnWithImgFc = stnFCwPolygons.map(calcNimg).filter(ee.Filter.gt('nImg',0)).map(calc_RrsMeanStd) \
        .map(calcRrsIn).filter(ee.Filter.eq('RrsIn',True)).map(add_product_id)
    if (stnWithImgFc.size().getInfo() > 0):
        stnWithImgDf = geemap.ee_to_pandas(stnWithImgFc, col_names=colNames)
        stnWithImgDf = stnWithImgDf.dropna().reset_index(drop=True)
        rrs_df = pd.concat([rrs_df,stnWithImgDf])
        print('    Found: %d images; Rrs calculated for %d stations' % \
                      (eeImgColl.size().getInfo(), len(stnWithImgDf)))
    else:
        print('    Found: %d images; Rrs calculated for 0 stations' % eeImgColl.size().getInfo())
    indx_start = indx_end; indx_end = indx_start + nStn2process
print('Done!')
# Export to CSV file 
if (not os.path.exists(save_dir)):
    os.mkdir(save_dir)
rrs_df.to_csv(os.path.join(save_dir,save_fn), index=False)

**<em>Landsat 5 (L5) was operational from 16 March 1984 to 5 May 2012.</em>** 

In [ ]:
# Set the satellite code and start and end dates
sat_code = 'L5'; date_start = '1984-03-16'; date_end = '2012-05-06'
stn_dfSat = stn_df.loc[(stn_df['date']>=pd.to_datetime(date_start))& \
                     (stn_df['date']<=pd.to_datetime(date_end))].reset_index(drop=True).copy()
nStn = len(stn_dfSat)
# Subdirectory and file names
save_fn = 'L5_Rrs0010C.csv'
# Copy geodatabase with columns of interest 
ccc_ponds_gdf1 = ccc_ponds_gdf[['CCC_GIS_ID','geometry']].copy()
# Set satellite parameters
ee_product,bands_list,qa_bands_list,img_scale,img_offset,scale_m,bands4rgb_dict,save_dir = set_satellite(sat_code)
colNames = ['PRODUCT_ID','CCC_GIS_ID','day_str','SDD (m)','lagDays_mean','lagDays_stdDev','latitude','longitude'] \
    + list(np.array([[s+'_mean',s+'_stdDev'] for s in bands_list]).flatten())
# Extract Rrs
indx_start = 0; indx_end = indx_start+nStn2process
rrs_df = pd.DataFrame()
while (indx_start<nStn):
    stn_df1 = stn_dfSat[indx_start:indx_end].copy()
    stn_gdf = gpd.GeoDataFrame(stn_df1.merge(ccc_ponds_gdf1, how='left', left_on='CCC_GIS_ID', \
                                        right_on='CCC_GIS_ID').dropna())
    print('Processing observations %d - %d of %d (%d stations)' % (indx_start, indx_end, nStn, len(stn_gdf)))
    # Create feature collection
    stnFCwPolygons = geemap.geopandas_to_ee(stn_gdf, date='day_str', date_format='YYYY-MM-dd')
    eeImgColl = ee.ImageCollection(ee_product) \
      .filterDate(ee.Date(str(np.min(stn_gdf['date']).strftime('%Y-%m-%d'))).advance(-delta_days,'day'), \
              ee.Date(str(np.max(stn_gdf['date']).strftime('%Y-%m-%d'))).advance(delta_days,'day')) \
      .filterBounds(stnFCwPolygons) \
      .sort("system:time_start") \
      .select(bands_list + qa_bands_list)
    # Apply cloud mask 
    imgCollMasked = eeImgColl.map(lambda image: img_mask_L5789(image)).select(bands_list) \
        .map(lambda image: image.multiply(img_scale).add(img_offset).copyProperties(image, ['system:time_start']))
    # Generate a List to compare dates
    eeImgCollList = eeImgColl.toList(eeImgColl.size())
    image = ee.Image(eeImgCollList.get(0))
    # Add in the end of the list a dummy image
    eeImgCollList = eeImgCollList.add(image)
    stnWithImgFc = stnFCwPolygons.map(calcNimg).filter(ee.Filter.gt('nImg',0)).map(calc_RrsMeanStd) \
        .map(calcRrsIn).filter(ee.Filter.eq('RrsIn',True)).map(add_product_id)
    if (stnWithImgFc.size().getInfo() > 0):
        stnWithImgDf = geemap.ee_to_pandas(stnWithImgFc, col_names=colNames)
        stnWithImgDf = stnWithImgDf.dropna().reset_index(drop=True)
        rrs_df = pd.concat([rrs_df,stnWithImgDf])
        print('    Found: %d images; Rrs calculated for %d stations' % \
                      (eeImgColl.size().getInfo(), len(stnWithImgDf)))
    else:
        print('    Found: %d images; Rrs calculated for 0 stations' % eeImgColl.size().getInfo())
    indx_start = indx_end; indx_end = indx_start + nStn2process
print('Done!')
# Export to CSV file 
if (not os.path.exists(save_dir)):
    os.mkdir(save_dir)
rrs_df.to_csv(os.path.join(save_dir,save_fn), index=False)

**<em>Landsat 7 (L7) was operational from 28 May 1999 to 6 April 2022.</em>**

In [ ]:
# Set the satellite code and start and end dates
sat_code = 'L7'; date_start = '1999-05-28'; date_end = '2024-01-01'
stn_dfSat = stn_df.loc[(stn_df['date']>=pd.to_datetime(date_start))& \
                     (stn_df['date']<=pd.to_datetime(date_end))].reset_index(drop=True).copy()
nStn = len(stn_dfSat)
# Subdirectory and file names
save_fn = 'L7_Rrs0010C.csv'
# Copy geodatabase with columns of interest 
ccc_ponds_gdf1 = ccc_ponds_gdf[['CCC_GIS_ID','geometry']].copy()
# Set satellite parameters
ee_product,bands_list,qa_bands_list,img_scale,img_offset,scale_m,bands4rgb_dict,save_dir = set_satellite(sat_code)
colNames = ['PRODUCT_ID','CCC_GIS_ID','day_str','SDD (m)','lagDays_mean','lagDays_stdDev','latitude','longitude'] \
    + list(np.array([[s+'_mean',s+'_stdDev'] for s in bands_list]).flatten())
# Extract Rrs
indx_start = 0; indx_end = indx_start+nStn2process
rrs_df = pd.DataFrame()
while (indx_start<nStn):
    stn_df1 = stn_dfSat[indx_start:indx_end].copy()
    stn_gdf = gpd.GeoDataFrame(stn_df1.merge(ccc_ponds_gdf1, how='left', left_on='CCC_GIS_ID', \
                                        right_on='CCC_GIS_ID').dropna())
    print('Processing observations %d - %d of %d (%d stations)' % (indx_start, indx_end, nStn, len(stn_gdf)))
    # Create feature collection
    stnFCwPolygons = geemap.geopandas_to_ee(stn_gdf, date='day_str', date_format='YYYY-MM-dd')
    eeImgColl = ee.ImageCollection(ee_product) \
      .filterDate(ee.Date(str(np.min(stn_gdf['date']).strftime('%Y-%m-%d'))).advance(-delta_days,'day'), \
              ee.Date(str(np.max(stn_gdf['date']).strftime('%Y-%m-%d'))).advance(delta_days,'day')) \
      .filterBounds(stnFCwPolygons) \
      .sort("system:time_start") \
      .select(bands_list + qa_bands_list)
    # Apply cloud mask 
    imgCollMasked = eeImgColl.map(lambda image: img_mask_L5789(image)).select(bands_list) \
        .map(lambda image: image.multiply(img_scale).add(img_offset).copyProperties(image, ['system:time_start']))
    # Generate a List to compare dates
    eeImgCollList = eeImgColl.toList(eeImgColl.size())
    image = ee.Image(eeImgCollList.get(0))
    # Add in the end of the list a dummy image
    eeImgCollList = eeImgCollList.add(image)
    stnWithImgFc = stnFCwPolygons.map(calcNimg).filter(ee.Filter.gt('nImg',0)).map(calc_RrsMeanStd) \
        .map(calcRrsIn).filter(ee.Filter.eq('RrsIn',True)).map(add_product_id)
    if (stnWithImgFc.size().getInfo() > 0):
        stnWithImgDf = geemap.ee_to_pandas(stnWithImgFc, col_names=colNames)
        stnWithImgDf = stnWithImgDf.dropna().reset_index(drop=True)
        rrs_df = pd.concat([rrs_df,stnWithImgDf])
        print('    Found: %d images; Rrs calculated for %d stations' % \
                      (eeImgColl.size().getInfo(), len(stnWithImgDf)))
    else:
        print('    Found: %d images; Rrs calculated for 0 stations' % eeImgColl.size().getInfo())
    indx_start = indx_end; indx_end = indx_start + nStn2process
print('Done!')
# Export to CSV file 
if (not os.path.exists(save_dir)):
    os.mkdir(save_dir)
rrs_df.to_csv(os.path.join(save_dir,save_fn), index=False)

**<em>Landsat 8 (L8) which was launched 18 March 2013 and is still operational.</em>**

In [ ]:
# Set the satellite code and start and end dates
sat_code = 'L8'; date_start = '2013-03-18'; date_end = '2024-01-01'
stn_dfSat = stn_df.loc[(stn_df['date']>=pd.to_datetime(date_start))& \
                     (stn_df['date']<=pd.to_datetime(date_end))].reset_index(drop=True).copy()
nStn = len(stn_dfSat)
# Subdirectory and file names
save_fn = 'L8_Rrs0010C.csv'
# Copy geodatabase with columns of interest 
ccc_ponds_gdf1 = ccc_ponds_gdf[['CCC_GIS_ID','geometry']].copy()
# Set satellite parameters
ee_product,bands_list,qa_bands_list,img_scale,img_offset,scale_m,bands4rgb_dict,save_dir = set_satellite(sat_code)
colNames = ['PRODUCT_ID','CCC_GIS_ID','day_str','SDD (m)','lagDays_mean','lagDays_stdDev','latitude','longitude'] \
    + list(np.array([[s+'_mean',s+'_stdDev'] for s in bands_list]).flatten())
# Extract Rrs
indx_start = 0; indx_end = indx_start+nStn2process
rrs_df = pd.DataFrame()
while (indx_start<nStn):
    stn_df1 = stn_dfSat[indx_start:indx_end].copy()
    stn_gdf = gpd.GeoDataFrame(stn_df1.merge(ccc_ponds_gdf1, how='left', left_on='CCC_GIS_ID', \
                                        right_on='CCC_GIS_ID').dropna())
    print('Processing observations %d - %d of %d (%d stations)' % (indx_start, indx_end, nStn, len(stn_gdf)))
    # Create feature collection
    stnFCwPolygons = geemap.geopandas_to_ee(stn_gdf, date='day_str', date_format='YYYY-MM-dd')
    eeImgColl = ee.ImageCollection(ee_product) \
      .filterDate(ee.Date(str(np.min(stn_gdf['date']).strftime('%Y-%m-%d'))).advance(-delta_days,'day'), \
              ee.Date(str(np.max(stn_gdf['date']).strftime('%Y-%m-%d'))).advance(delta_days,'day')) \
      .filterBounds(stnFCwPolygons) \
      .sort("system:time_start") \
      .select(bands_list + qa_bands_list)
    # Apply cloud mask 
    imgCollMasked = eeImgColl.map(lambda image: img_mask_L5789(image)).select(bands_list) \
        .map(lambda image: image.multiply(img_scale).add(img_offset).copyProperties(image, ['system:time_start']))
    # Generate a List to compare dates
    eeImgCollList = eeImgColl.toList(eeImgColl.size())
    image = ee.Image(eeImgCollList.get(0))
    # Add in the end of the list a dummy image
    eeImgCollList = eeImgCollList.add(image)
    stnWithImgFc = stnFCwPolygons.map(calcNimg).filter(ee.Filter.gt('nImg',0)).map(calc_RrsMeanStd) \
        .map(calcRrsIn).filter(ee.Filter.eq('RrsIn',True)).map(add_product_id)
    if (stnWithImgFc.size().getInfo() > 0):
        stnWithImgDf = geemap.ee_to_pandas(stnWithImgFc, col_names=colNames)
        stnWithImgDf = stnWithImgDf.dropna().reset_index(drop=True)
        rrs_df = pd.concat([rrs_df,stnWithImgDf])
        print('    Found: %d images; Rrs calculated for %d stations' % \
                      (eeImgColl.size().getInfo(), len(stnWithImgDf)))
    else:
        print('    Found: %d images; Rrs calculated for 0 stations' % eeImgColl.size().getInfo())
    indx_start = indx_end; indx_end = indx_start + nStn2process
print('Done!')
# Export to CSV file 
if (not os.path.exists(save_dir)):
    os.mkdir(save_dir)
rrs_df.to_csv(os.path.join(save_dir,save_fn), index=False)

**<em>Landsat 9 (L9) which was launched 31 October 2021 and is still operational.</em>** 

In [ ]:
# Set the satellite code and start and end dates
sat_code = 'L9'; date_start = '2021-10-31'; date_end = '2024-01-01'
stn_dfSat = stn_df.loc[(stn_df['date']>=pd.to_datetime(date_start))& \
                     (stn_df['date']<=pd.to_datetime(date_end))].reset_index(drop=True).copy()
nStn = len(stn_dfSat)
# Subdirectory and file names
save_fn = 'L9_Rrs0010C.csv'
# Copy geodatabase with columns of interest 
ccc_ponds_gdf1 = ccc_ponds_gdf[['CCC_GIS_ID','geometry']].copy()
# Set satellite parameters
ee_product,bands_list,qa_bands_list,img_scale,img_offset,scale_m,bands4rgb_dict,save_dir = set_satellite(sat_code)
colNames = ['PRODUCT_ID','CCC_GIS_ID','day_str','SDD (m)','lagDays_mean','lagDays_stdDev','latitude','longitude'] \
    + list(np.array([[s+'_mean',s+'_stdDev'] for s in bands_list]).flatten())
# Extract Rrs
indx_start = 0; indx_end = indx_start+nStn2process
rrs_df = pd.DataFrame()
while (indx_start<nStn):
    stn_df1 = stn_dfSat[indx_start:indx_end].copy()
    stn_gdf = gpd.GeoDataFrame(stn_df1.merge(ccc_ponds_gdf1, how='left', left_on='CCC_GIS_ID', \
                                        right_on='CCC_GIS_ID').dropna())
    print('Processing observations %d - %d of %d (%d stations)' % (indx_start, indx_end, nStn, len(stn_gdf)))
    # Create feature collection
    stnFCwPolygons = geemap.geopandas_to_ee(stn_gdf, date='day_str', date_format='YYYY-MM-dd')
    eeImgColl = ee.ImageCollection(ee_product) \
      .filterDate(ee.Date(str(np.min(stn_gdf['date']).strftime('%Y-%m-%d'))).advance(-delta_days,'day'), \
              ee.Date(str(np.max(stn_gdf['date']).strftime('%Y-%m-%d'))).advance(delta_days,'day')) \
      .filterBounds(stnFCwPolygons) \
      .sort("system:time_start") \
      .select(bands_list + qa_bands_list)
    # Apply cloud mask 
    imgCollMasked = eeImgColl.map(lambda image: img_mask_L5789(image)).select(bands_list) \
        .map(lambda image: image.multiply(img_scale).add(img_offset).copyProperties(image, ['system:time_start']))
    # Generate a List to compare dates
    eeImgCollList = eeImgColl.toList(eeImgColl.size())
    image = ee.Image(eeImgCollList.get(0))
    # Add in the end of the list a dummy image
    eeImgCollList = eeImgCollList.add(image)
    stnWithImgFc = stnFCwPolygons.map(calcNimg).filter(ee.Filter.gt('nImg',0)).map(calc_RrsMeanStd) \
        .map(calcRrsIn).filter(ee.Filter.eq('RrsIn',True)).map(add_product_id)
    if (stnWithImgFc.size().getInfo() > 0):
        stnWithImgDf = geemap.ee_to_pandas(stnWithImgFc, col_names=colNames)
        stnWithImgDf = stnWithImgDf.dropna().reset_index(drop=True)
        rrs_df = pd.concat([rrs_df,stnWithImgDf])
        print('    Found: %d images; Rrs calculated for %d stations' % \
                      (eeImgColl.size().getInfo(), len(stnWithImgDf)))
    else:
        print('    Found: %d images; Rrs calculated for 0 stations' % eeImgColl.size().getInfo())
    indx_start = indx_end; indx_end = indx_start + nStn2process
print('Done!')
# Export to CSV file 
if (not os.path.exists(save_dir)):
    os.mkdir(save_dir)
rrs_df.to_csv(os.path.join(save_dir,save_fn), index=False)